# Cats and Dogs (MLP and CNN Code):-


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the dataset with split and metadata
(train_ds, val_ds), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,  # returns (image, label)
)

# Resize and batch
IMG_SIZE = 160
BATCH_SIZE = 32

def format_example(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = train_ds.map(format_example).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.map(format_example).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.IWIXTF_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf


In [ ]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# train_datagen = ImageDataGenerator(
#     rescale = 1./255,
#     shear_range = 0.2,
#     zoom_range = 0.2,
#     horizontal_flip = True,
#     width_shift_range = 0.2,
#     height_shift_range = 0.2,
#     rotation_range = 40,
#     fill_mode = 'nearest'
# )

# train_gen = train_datagen.flow_from_directory(
#     train_dir,
#     target_size = (150 , 150),
#     batch_size = 32,
#     class_mode = 'binary'
# )

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D , MaxPooling2D , Flatten , Dense ,Dropout , BatchNormalization
from keras.callbacks import EarlyStopping , ModelCheckpoint , ReduceLROnPlateau

model = Sequential([
    Conv2D(32,(3,3), activation = 'relu' , input_shape = (160 , 160 , 3)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.25),

    Conv2D(64 , (3 , 3) , activation = 'relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.25), # Added comma here

    Conv2D(128 , (3 , 3) , activation = 'relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.25), # Added comma here

    Flatten(),
    Dense(512 , activation = 'relu'),
    BatchNormalization(),
    Dropout(0.25),
    Dense(1 , activation = 'sigmoid')  #Binary Output
])

model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy']) # Changed loss to binary_crossentropy for binary classification

In [ ]:
from keras.callbacks import EarlyStopping , ModelCheckpoint , ReduceLROnPlateau

early_stop = EarlyStopping(monitor = 'val_loss' , patience = 5 , verbose = 1 , restore_best_weights = True)
checkpoint = ModelCheckpoint('best_cat_dog_model.h5' , monitor = 'val_accuracy' ,save_best_only = True )

In [ ]:
from keras.callbacks import EarlyStopping , ModelCheckpoint , ReduceLROnPlateau

# Assuming early_stop and checkpoint callbacks are defined in a previous cell
# You can combine them into a list
callbacks_list = [early_stop, checkpoint]

model.fit(train_ds , epochs =  5, callbacks = callbacks_list , validation_data=val_ds) # Corrected epoches to epochs, callback to callbacks, and added validation_data

Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.6482 - loss: 0.7553

582/582 ━━━━━━━━━━━━━━━━━━━━ 52s 69ms/step - accuracy: 0.6482 - loss: 0.7551 - val_accuracy: 0.7317 - val_loss: 0.5443
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.7611 - loss: 0.4963

582/582 ━━━━━━━━━━━━━━━━━━━━ 68s 59ms/step - accuracy: 0.7612 - loss: 0.4963 - val_accuracy: 0.7949 - val_loss: 0.4391
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 35s 48ms/step - accuracy: 0.8111 - loss: 0.4149 - val_accuracy: 0.6836 - val_loss: 0.6975
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 29s 49ms/step - accuracy: 0.8392 - loss: 0.3649 - val_accuracy: 0.7898 - val_loss: 0.4926
Epoch 5/5
581/582 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.8662 - loss: 0.3132

582/582 ━━━━━━━━━━━━━━━━━━━━ 65s 112ms/step - accuracy: 0.8662 - loss: 0.3132 - val_accuracy: 0.7992 - val_loss: 0.5233
Restoring model weights from the end of the best epoch: 2.


In [ ]:
model.evaluate(train_ds)

582/582 ━━━━━━━━━━━━━━━━━━━━ 21s 37ms/step - accuracy: 0.8132 - loss: 0.4165


[0.39901071786880493, 0.8222460746765137]

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Replace this path with the actual path to your image file in Google Colab
image_path = '/content/1.jpg'

img = image.load_img(image_path, target_size=(160, 160))
img_array = image.img_to_array(img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)


prediction = model.predict(img_array)

if prediction[0][0] >= 0.5:
    print("🐶 It's a dog!")
else:
    print("🐱 It's a cat!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
🐱 It's a cat!
